In [1]:
import time
import pickle
import random
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from model import CRAB_Model
seed_value = 12345678
random.seed(seed_value)
np.random.seed(seed=seed_value)
import gdxpds

Especially on Linux, gdxpds should be imported before pandas to avoid a library conflict. Also make sure your GAMS directory is listed in LD_LIBRARY_PATH.


In [2]:
dataframes = gdxpds.to_dataframes('results_ssp2_cba_noncoop_crab_in.gdx')

In [3]:
path_inputs_crab = "CRAB/Inputs/"
# load model inputs
socio_economic_hh_attributes = pd.read_csv(path_inputs_crab + "HH_attributes_us")
rice_step = 1

In [4]:
socio_economic_hh_attributes

,Unnamed: 0,Flood_prob,Worry,RE_elev,RE_wet,RE_dry,SE_elev,SE_wet,SE_dry,PC_elev,...,Social network,Education,Savings_norm,HH_sell_norm,t,OMEGA,CPC,Y,I,C
0,0,0.10,3.0,3.0,2.000000,1.333333,1.0,1.000000,1.000000,5.0,...,1.0,2.0,0.168,8.700181,2,NaN,39.249089,NaN,3.158033,NaN
1,1,0.20,3.0,3.0,3.000000,3.000000,3.0,3.000000,3.000000,3.0,...,6.0,3.0,0.206,29.000603,2,NaN,39.249089,NaN,3.158033,NaN
2,2,0.05,1.0,5.0,5.000000,5.000000,5.0,5.000000,5.000000,5.0,...,0.0,4.0,4.092,34.800723,2,NaN,39.249089,NaN,3.158033,NaN
3,3,0.20,1.0,3.0,4.666667,3.000000,1.0,1.000000,1.000000,5.0,...,0.0,3.0,0.000,17.400362,2,NaN,39.249089,NaN,3.158033,NaN
4,4,1.00,2.0,1.0,3.666667,3.333333,1.0,4.000000,4.000000,5.0,...,0.0,3.0,0.683,14.500301,2,NaN,39.249089,NaN,3.158033,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,0.30,3.0,4.0,4.000000,4.000000,1.0,3.000000,2.000000,4.0,...,3.0,4.0,2.799,43.500904,2,NaN,39.249089,NaN,3.158033,NaN
4996,4996,0.20,2.0,5.0,4.333333,4.000000,5.0,4.333333,4.000000,5.0,...,2.0,3.0,2.256,23.200482,2,NaN,39.249089,NaN,3.158033,NaN
4997,4997,0.00,2.0,1.0,4.000000,3.666667,1.0,1.000000,1.000000,5.0,...,3.0,2.0,0.000,11.600241,2,NaN,39.249089,NaN,3.158033,NaN
4998,4998,0.01,3.0,4.0,4.666667,4.333333,3.0,4.333333,4.333333,4.0,...,2.0,3.0,1.022,24.360506,2,NaN,39.249089,NaN,3.158033,NaN


In [8]:


# get t to be used in the ABM
t = socio_economic_hh_attributes['t'][0]
#print('t is ddddd', t)
# make list of variable of interests (omega, consumption, investment, expected damages etc) + for the regions of interest
# iterate over them to get  values of interest
# put together in a dataframe
input_list = []
variable_of_interest = ['OMEGA', 'C', 'Y']
regions_of_interest = ['us']
for i in variable_of_interest:
    dataframe = dataframes[i]
    dataframe['t'] = dataframe['t'].astype(int)
    for j in regions_of_interest:
        dataframe = dataframe.loc[(dataframe['t'] == t) & (dataframe['n'] == 'usa')]
        dataframe['variable'] = i
        input_list.append(dataframe)

/var/folders/33/dr4cqnzj0597_7n7kbsrfht8cdq0jk/T/ipykernel_56822/1652757757.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['variable'] = i
/var/folders/33/dr4cqnzj0597_7n7kbsrfht8cdq0jk/T/ipykernel_56822/1652757757.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['variable'] = i
/var/folders/33/dr4cqnzj0597_7n7kbsrfht8cdq0jk/T/ipykernel_56822/1652757757.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [9]:
input_list = pd.concat(input_list)

In [10]:
input_list

,t,n,Level,Marginal,Lower,Upper,Scale,variable
54,2,usa,0.001135,0.0,4.000000e+300,3.000000e+300,1.0,OMEGA
111,2,usa,12.933791,0.0,1.000000e-08,3.000000e+300,1.0,C
111,2,usa,16.092273,0.0,0.000000e+00,3.000000e+300,1.0,Y


In [11]:


# add 'Level' from input_list to socio_economic_hh_attributea, for each variable of interest
for i in variable_of_interest:
    variable = input_list.loc[input_list['variable'] == i]['Level']
    socio_economic_hh_attributes[i] = variable.mean()


In [12]:
socio_economic_hh_attributes

,Unnamed: 0,Flood_prob,Worry,RE_elev,RE_wet,RE_dry,SE_elev,SE_wet,SE_dry,PC_elev,...,Social network,Education,Savings_norm,HH_sell_norm,t,OMEGA,CPC,Y,I,C
0,0,0.10,3.0,3.0,2.000000,1.333333,1.0,1.000000,1.000000,5.0,...,1.0,2.0,0.168,8.700181,2,0.001135,39.249089,16.092273,3.158033,12.933791
1,1,0.20,3.0,3.0,3.000000,3.000000,3.0,3.000000,3.000000,3.0,...,6.0,3.0,0.206,29.000603,2,0.001135,39.249089,16.092273,3.158033,12.933791
2,2,0.05,1.0,5.0,5.000000,5.000000,5.0,5.000000,5.000000,5.0,...,0.0,4.0,4.092,34.800723,2,0.001135,39.249089,16.092273,3.158033,12.933791
3,3,0.20,1.0,3.0,4.666667,3.000000,1.0,1.000000,1.000000,5.0,...,0.0,3.0,0.000,17.400362,2,0.001135,39.249089,16.092273,3.158033,12.933791
4,4,1.00,2.0,1.0,3.666667,3.333333,1.0,4.000000,4.000000,5.0,...,0.0,3.0,0.683,14.500301,2,0.001135,39.249089,16.092273,3.158033,12.933791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,0.30,3.0,4.0,4.000000,4.000000,1.0,3.000000,2.000000,4.0,...,3.0,4.0,2.799,43.500904,2,0.001135,39.249089,16.092273,3.158033,12.933791
4996,4996,0.20,2.0,5.0,4.333333,4.000000,5.0,4.333333,4.000000,5.0,...,2.0,3.0,2.256,23.200482,2,0.001135,39.249089,16.092273,3.158033,12.933791
4997,4997,0.00,2.0,1.0,4.000000,3.666667,1.0,1.000000,1.000000,5.0,...,3.0,2.0,0.000,11.600241,2,0.001135,39.249089,16.092273,3.158033,12.933791
4998,4998,0.01,3.0,4.0,4.666667,4.333333,3.0,4.333333,4.333333,4.0,...,2.0,3.0,1.022,24.360506,2,0.001135,39.249089,16.092273,3.158033,12.933791


In [ ]:
runs = 10
steps = 300
#seeds = [69593830] 
seeds = [random.randint(1, seed_value ) for i in range(runs) ]

In [ ]:
micro_variables = []
macro_variables = []
for i, seed in enumerate(seeds):
    # print(("Model initialized with " + str(H) + " Households"))
    print("Run num", i, "with random seed", seed)
    tic = time.time()
    model = CRAB_Model(F1=125, F2=200, F3=300,H=5000, Exp = 500,RCP = 2.5, seed=seed, hh_cca = False,  insurance_hh= False, floods=False)
    model.reset_randomizer(seed)
    for j in range(steps):
        print("#------------ step", j+1, "------------#")
        model.step()
    toc = time.time()
    runtime = toc - tic
    # runtimes.append(runtime)
    print("MODEL runtime: " + str(runtime))
    print()
    # # --------
    # # COMMENT:
    # # Look into efficient output saving later
    # # --------
    macro_variable = model.datacollector.get_model_vars_dataframe()
    # # Iteratively add dataframe to list
    macro_variables.append(macro_variable)
    micro_variable = model.datacollector.get_agent_vars_dataframe()
    micro_variables.append(micro_variable)
    #micro_variable = micro_variable.dropna()
    # # micro_variable[["Demand coastal", "Demand Inland"]] = pd.DataFrame(micro_variable.Real_demand_cap.to_list(), index=micro_variable.index)
    # # micro_variable[["Competitiveness region 0", "Competitiveness region 1", "Competitiveness export"]] = pd.DataFrame(micro_variable.Competitiveness.to_list(), index=micro_variable.index)
    # # micro_variable[["MS region 0", "MS region 1", "MS export"]] = pd.DataFrame(micro_variable.Ms.to_list(), index=micro_variable.index)
    # # micro_variable[["Prod A", "Prod B"]] = pd.DataFrame(micro_variable.Prod.to_list(), index=micro_variable.index)

In [ ]:
macro_variable = model.datacollector.get_model_vars_dataframe()
# # Iteratively add dataframe to list
macro_variables.append(macro_variable)
micro_variable = model.datacollector.get_agent_vars_dataframe()
#micro_variable = micro_variable.dropna()

# # micro_variable[["Demand coastal", "Demand Inland"]] = pd.DataFrame(micro_variable.Real_demand_cap.to_list(), index=micro_variable.index)
# # micro_variable[["Competitiveness region 0", "Competitiveness region 1", "Competitiveness export"]] = pd.DataFrame(micro_variable.Competitiveness.to_list(), index=micro_variable.index)
# # micro_variable[["MS region 0", "MS region 1", "MS export"]] = pd.DataFrame(micro_variable.Ms.to_list(), index=micro_variable.index)
# # micro_variable[["Prod A", "Prod B"]] = pd.DataFrame(micro_variable.Prod.to_list(), index=micro_variable.index)
micro_variables.append(micro_variable)

In [ ]:
'''
experiments = []
dr_cca = micro_variable
dr_cca['CCA'] = 'DR'
experiments.append(dr_cca)
dr_ins = micro_variable
dr_ins['CCA'] = 'Ins'
micro_variables.append(dr_cca)
no_cca = micro_variable
no_cca['CCA'] = 'None'
experiments.append(no_cca)
df_cons = pd.concat(experiments).reset_index()
df_cons['CCA'].value_counts()
'''

In [ ]:
macro_variable['HH_in_out'] = macro_variable['HH_change'] * 0.5 * macro_variable['Households Coastal region']

In [ ]:
'''
macro_variable["LD_cap_0"].plot()
macro_variable["LD_cons_0"].plot()
macro_variable["LD_serv_0"].plot()
micro_variable.reset_index().loc[micro_variable['Step'] == 230]['Total_damage'].sum()
micro_variable.reset_index().groupby(['AgentID'])['Repair_exp'].max().sum()
'''

In [ ]:
#macro_variable['HH_change'].plot()

In [ ]:
#np.log(macro_variable['Av_income_pp'].iloc[10:]).pct_change().plot()

In [ ]:
#macro_variable['HH_in_out'].plot()

In [ ]:
df_cons_hh = micro_variable[micro_variable['Type'] == 'Household']

In [ ]:
df_cons_hh['number'] =1

In [ ]:
#sns.lineplot(data = df_cons_hh, x='Step', y= 'number', estimator='sum' ,hue='CCA')

In [ ]:
sns.lineplot(data = df_cons_hh, x='Step', y= 'number', estimator='sum')

In [ ]:
micro_variable['number'] = 1

In [ ]:
micro_variable

In [ ]:
grouped_average = micro_variable.groupby(['Step'])['Wage','Net_worth','Price' ].mean().reset_index()
grouped_sum = micro_variable.groupby(['Step'])['Production_made','Investment', 'number' , 'Cons' ].sum().reset_index()

In [ ]:
grouped_average

In [ ]:
grouped_sum

In [ ]:
price = np.log(grouped_average['Price'])
wage = np.log(grouped_average['Wage'])
infl = np.log(grouped_average['Price'].pct_change())
inv = np.log(grouped_sum['Investment'])
cons = np.log(grouped_sum['Cons'])
output = np.log(grouped_sum['Production_made'])
unem = np.log(macro_variable['Unemployment rate coastal'])
corr_list  = [  unem, infl,  price , inv , cons ,output ]
df_corr = pd.concat(corr_list, axis = 1)
df_corr.columns = [  'Unemployment', 'Infl.','Price',  'Inv',  'Cons','Output']
#corrMatrix = df_corr.corr(method='pearson')
corrMatrix = df_corr.corr(method='pearson')
sns.heatmap(corrMatrix, annot=True)
plt.show()
fig, ax = plt.subplots(figsize=(6, 6)) 
mask = np.zeros_like(corrMatrix.corr())
mask[np.triu_indices_from(mask)] = 1
sns.heatmap(corrMatrix, mask= mask, ax= ax, annot= True)

In [ ]:

import statsmodels.api as sm
gdp = np.log(grouped_sum['Production_made'] + 1)
inv = np.log(grouped_sum['Investment'] + 1)
cons =  np.log(grouped_sum['Cons'] + 1)

bp = pd.concat([gdp, inv, cons], axis = 1)
bp = bp.iloc[50:]
bp.columns = ['GDP', 'INV', 'CONS']
print(bp)
#cycles = sm.tsa.filters.bkfilter(bp[['INV','CONS', 'GDP']], 6, 32, 12)
cycles = sm.tsa.filters.bkfilter(bp[['INV','CONS', 'GDP']], 6, 32, 12)

cycles, cf_trend = sm.tsa.filters.cffilter(bp[['INV','CONS', 'GDP']])
# / abs(cycles['INV_cycle'].mean())

cycles.columns = ['Investment', 'Consumption', 'Output']
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)
cycles.plot(ax=ax, style=['r--','b-'])
plt.legend(fontsize = 16)
plt.show()

In [ ]:
std_bp_gdp = cycles['Output'].std() #/ abs(cycles['GDP_cycle'].mean()) 
std_bp_cons =  cycles['Consumption'].std()# / abs(cycles['CONS_cycle'].mean())
std_bp_inv =  cycles['Investment'].std()
std_bp_gdp, std_bp_cons, std_bp_inv

In [ ]:
sns.lineplot(data = df_cons_hh, x='Step', y= 'Wage')

In [ ]:
'''

df_cons_hh['Total_expenditure'] = df_cons_hh['Repair_exp'] + df_cons_hh['Cons']
sns.lineplot(data = df_cons, x = 'Step', y = 'Production_made', hue = 'CCA')
sns.lineplot(data = df_cons_hh, x = 'Step', y = 'Total_expenditure', hue = 'CCA')
sns.lineplot(data = df_cons_hh, x = 'Step', y = 'Net_worth', hue = 'CCA')
'''

In [ ]:
macro_variable.iloc[50:].loc[macro_variable['Unemployment rate coastal'] == 0]

In [ ]:
macro_variable['Unemployment rate coastal'].plot()

In [ ]:
macro_variable['Unemployment rate coastal']  



In [ ]:
results_path = "/Users/ataberna/Documents/IIASA/Results"

In [ ]:
micro_df = pd.concat(micro_variables)
micro_df = micro_df.reset_index(level=["Step"])
micro_df = micro_df.reset_index(level= 'AgentID')
micro_df = micro_df.reset_index()
#micro_df.to_csv(results_path + "/micro_variables")
macro = pd.concat(macro_variables)
#macro_variable.to_csv(results_path + "/macro_variables")


In [ ]:
firms_df = micro_df.loc[micro_df['Type'] != 'Household']

In [ ]:
sns.histplot(data = firms_df.loc[firms_df['Step'] == 300], x = 'Size', hue = 'Type')

In [ ]:
firms_df.loc[firms_df['Step'] == 30]['Size'].value_counts()

In [ ]:
firms_df

In [ ]:
step_300_df_cap = firms_df.loc[(firms_df['Step'] == 250) & (firms_df['Type'] == 'Cap')]

In [ ]:
#step_300_df_cap.loc[step_300_df_cap['Price'] == step_300_df_cap['Price'].max()]
#step_300_df_cap['Price_q'] = step_300_df_cap['Price'] * step_300_df_cap['Production_made']
#step_300_df_cap['Price'].hist()

In [ ]:
#sns.lineplot(data= firms_df, x = 'Step', y = 'Price', hue = 'Type')

In [ ]:
sns.lineplot(data= micro_df, x = 'Step', y = 'Production_made', hue = 'Type',  estimator='sum')

In [ ]:
firms_df['number'] = 1 

In [ ]:
cca_df_sub_floods_grouped_sum = firms_df.groupby(['Step', 'Type'])['Production_made','Net_worth', 'number'].sum().reset_index()

In [ ]:
sns.lineplot(data= firms_df, x = 'Step', y = 'number',  style= 'Type', estimator='sum')


In [ ]:
sns.lineplot(data= firms_df, x = 'Step', y = 'Net_worth', hue = 'Type')

In [ ]:
micro_variable = model.datacollector.get_agent_vars_dataframe()

In [ ]:
micro_variable = micro_variable.reset_index()

In [ ]:
micro_variable.loc[(micro_variable['Type'] == 'Household') & (micro_variable['Step'] == 190)]['Insured'].value_counts()

In [ ]:
micro_variable.loc[(micro_variable['Type'] != 'Household') & (micro_variable['Step'] == 190)]['Insured'].value_counts()

In [ ]:
micro_df['number'] = 1
micro_variable['number'] = 1

In [ ]:
micro_variable_risk = micro_variable.loc[micro_variable['At_risk'] == True]

In [ ]:
final_micro_variable_risk = micro_variable_risk.loc[micro_variable_risk['Step'] == 190]

In [ ]:
tot_dry =  final_micro_variable_risk['Dry_proofed'].value_counts()
tot_dry


In [ ]:
tot_wet =  final_micro_variable_risk['Wet_proofed'].value_counts()
tot_wet


In [ ]:
tot_elev =  final_micro_variable_risk['Elevated'].value_counts()
tot_elev


In [ ]:
micro_variable_ins = micro_variable.loc[micro_variable['Premium'] > 0]

In [ ]:
final_micro_variable_risk

In [ ]:
tot_ins =  micro_variable_ins['Insured'].value_counts()
tot_ins


In [ ]:
tot_cca  ={ 'Insurance' :tot_ins[1] / (tot_ins[0] + tot_ins[1]),
            'Dry proofing' : tot_dry[1] / (tot_dry[0] + tot_dry[1]),
            'Wet proofing' : tot_wet[1] / (tot_wet[0] + tot_wet[1]),
            'Elevation' : tot_elev[1] / (tot_elev[0] + tot_elev[1])}

In [ ]:
tot_cca_df = pd.Series(tot_cca)

In [ ]:
labels = tot_cca_df.index

In [ ]:
plt.style.use(['science','no-latex','ieee'])

In [ ]:
tot_cca_df_no_sub = pd.read_csv('tot_cca_df_no_sub')
tot_cca_df_no_sub = tot_cca_df_no_sub.rename(columns = {'Unnamed: 0': 'CCA', '0': 'rate'})
tot_cca_df_no_sub = tot_cca_df_no_sub.set_index('CCA')
tot_cca_df_no_sub['CCA_sub'] = tot_cca_df

In [ ]:
tot_cca_df

In [ ]:
fig, axes = plt.subplots( 1,1, figsize = (4,4))
palette_1 = sns.color_palette("colorblind", 4)
labels = list(tot_cca_df.index)

sns.barplot(data = tot_cca_df_no_sub, x= list(tot_cca_df_no_sub.index), y = 'CCA_sub', color = 'gray', label= 'CCA uptake with subsidy')
sns.barplot(data = tot_cca_df_no_sub, x=  list(tot_cca_df_no_sub.index), y = 'rate', palette=palette_1 )
#axes.set_xticklabels( labels = list(reversed(tot_cca_df.index)) ,rotation=-1)
plt.legend()
axes.set_title('Expected sdaptation actions uptake in 2050')


In [ ]:
micro_variable_hh = micro_variable.loc[micro_variable['Type'] == 'Household']

In [ ]:
micro_variable_hh['Dry_proofed'].value_counts()[1] / (micro_variable_hh['Dry_proofed'].value_counts()[0] + micro_variable_hh['Dry_proofed'].value_counts()[1])

In [ ]:
micro_variable_hh['Wet_proofed'].value_counts()[1] / (micro_variable_hh['Wet_proofed'].value_counts()[0] + micro_variable_hh['Wet_proofed'].value_counts()[1])

In [ ]:
micro_variable_hh['Elevated'].value_counts()[1] / (micro_variable_hh['Elevated'].value_counts()[0] + micro_variable_hh['Elevated'].value_counts()[1])

In [ ]:
micro_variable_hh['Insured'].value_counts()

In [ ]:
#micro_variable.loc[micro_variable['Type'] != ['Household']]['Insured']

In [ ]:
micro_variable['Insured'].value_counts()[1] / (micro_variable['Insured'].value_counts()[0] + micro_variable['Insured'].value_counts()[1])

In [ ]:
micro_variables = micro_variable_hh.reset_index()

In [ ]:
micro_variables

In [ ]:
a = micro_variables.groupby(['Step'])['number','Elevated', 'Wet_proofed', 'Dry_proofed'].sum().reset_index()

In [ ]:
a['Ratio_elevated'] = a['Elevated'] / a['number']
a['Ratio_wet'] = a['Wet_proofed'] / a['number']
a['Ratio_dry'] = a['Dry_proofed'] / a['number']

In [ ]:
a

In [ ]:
sns.lineplot(data = a, x= 'Step', y = 'Ratio_elevated')
sns.lineplot(data = a, x= 'Step', y = 'Ratio_wet')
sns.lineplot(data = a, x= 'Step', y = 'Ratio_dry')


In [ ]:

#sns.lineplot(data= micro_df.loc[micro_df['Type'] != 'Household'], x = 'Step', y = 'number',  style= 'Type', estimator='sum')

In [ ]:
macro_variable['Unemployment rate coastal'].iloc[15:].plot()

In [ ]:
sns.lineplot(data= micro_df, x = 'Step', y = 'Production_made', hue = 'Type',  estimator='sum')

In [ ]:
sns.lineplot(data= micro_df, x = 'Step', y = 'Net_worth', hue = 'Type')
#sns.lineplot(data= micro_df.loc[micro_df['Type'] != 'Cap'], x = 'Step', y = 'Net_worth', hue = 'Type')

In [ ]:
micro_variable 
stop



In [ ]:
variables = ['At_risk', 'Education', 'Step', 'AgentID', 'Damage_coeff', 'House_value', 'Wage', 'Net worth','Monetary_damage' ]

In [ ]:
results_path =  "/Users/ataberna/Documents/PNAS/Results/test_homo"

In [ ]:
micro_variables_pmt_int_hom = pd.read_csv(results_path + '/micro_cca_2_floods_pmt_hom_t', usecols= variables)

In [ ]:
micro_variables_eu_hom = pd.read_csv(results_path + '/micro_2floods_eu_hom',  usecols= variables)

In [ ]:
micro_variables_no_cca_hom = pd.read_csv(results_path + '/micro_cca_2_floods_no_hom_t',  usecols= variables)

In [ ]:
micro_variables_eu_hom = micro_variables_pmt_int_hom

In [ ]:
micro_dfs_homo = [micro_variables_no_cca_hom, micro_variables_eu_hom, micro_variables_pmt_int_hom]

In [ ]:
micro_variables_pmt_int = pd.read_csv(results_path + '/micro_cca_2_floods_pmt_het_t',  usecols= variables)
micro_variables_pmt_int_no_floods = pd.read_csv(results_path + '/micro_pmt_cca_no_floods',  usecols= variables)
micro_variables_eu = pd.read_csv(results_path + '/micro_eu_cca_2_floods',  usecols= variables)
micro_variables_eu_no_floods = pd.read_csv(results_path + '/micro_eu_cca_no_floods',  usecols= variables)
micro_variables_no_cca = pd.read_csv(results_path + '/micro_no_cca_2_floods',  usecols= variables)
micro_variables_no_floods = pd.read_csv(results_path + '/micro_no_cca_no_floods',  usecols= variables)

In [ ]:
micro_dfs = [micro_variables_no_floods, micro_variables_no_cca,micro_variables_eu_no_floods ,micro_variables_eu, micro_variables_pmt_int_no_floods, micro_variables_pmt_int]

In [ ]:
micro_dfs[0]

In [ ]:
micro_dfs[5]['Education'].hist()

In [ ]:
theories = ['No adaptation', 'No adaptation', 'EU', 'EU', 'PMT', 'PMT']

In [ ]:
at_risk_df_list = []
at_risk_df_list_homo = []
homo = 0 
for i in range(len(micro_dfs)):
    at_risk_df =  micro_dfs[i].loc[(micro_dfs[i]['At_risk'] == True) & (micro_dfs[i]['Education'] != 1) & (micro_dfs[i]['Step'] >1 ) &  (micro_dfs[i]['AgentID'] % 10 == 0) ]# & (datasets[i]['Flooded'] == True)]
    at_risk_df['Theory'] = theories[i]
    if i % 2 == 0:
    
        at_risk_df_homo =  micro_dfs_homo[homo].loc[(micro_dfs_homo[homo]['At_risk'] == True) & (micro_dfs_homo[homo]['Education'] != 1) & (micro_dfs_homo[homo]['Step'] >1 ) &  (micro_dfs_homo[homo]['AgentID'] % 10 == 0) ]
        at_risk_df_homo['Theory'] = theories[homo]
        at_risk_df_list_homo.append(at_risk_df_homo)

        homo += 1
    #at_risk_df['Education'] = at_risk_df['Education'] - 1
    at_risk_df_list.append(at_risk_df)

In [ ]:
plt.style.use(['science','no-latex','ieee'])

In [ ]:
at_risk_df_list[1]['Education'].hist()

In [ ]:
linestyles = ['solid', (0, (3, 10, 1, 10)), 'dotted']
palette_1 = sns.color_palette("colorblind", 3)
font = 11
fig, axes = plt.subplots( 1,1, sharey = True, figsize = (7,7))
a =  0
for i in range(len(at_risk_df_list)):
    if i %2 == 0:

       legend = False
       sns.lineplot(at_risk_df_list[i + 1]['Step'], at_risk_df_list[i + 1]['Damage_coeff'], hue = at_risk_df_list[i + 1]['Education'], palette =palette_1, ci=95, linestyle = linestyles[a], legend=legend )

       a += 1
plt.plot(0,0, label = "Secondary", linestyle =linestyles[0], color = palette_1[0] )
plt.plot(0,0, label = "Tertiary", linestyle =linestyles[0], color = palette_1[1] )
plt.plot(0,0, label = "Post-grad", linestyle =linestyles[0], color = palette_1[2] )
plt.plot(0,0, label = "No adaptation - Heterogenous", linestyle =linestyles[0], color = 'black' )
plt.plot(0,0, label = "EU - Heterogenous", linestyle =linestyles[1], color = 'black' )
plt.plot(0,0, label = "Extended PMT - Heterogenous", linestyle =linestyles[2], color = 'black' )
plt.axvline(x = 100, linestyle = 'dashed')
plt.axvline(x = 140, linestyle = 'dashed')
plt.ylabel('Average damage coefficient', fontsize = font)
plt.xlabel('Step', fontsize = font)
plt.legend( loc = 'best', fontsize = font -1)
       #sns.lineplot(at_risk_df_list[i + 1]['Step'], at_risk_df_list[i + 1]['Elevated'], hue = at_risk_df_list[i + 1]['Education'], ci = None)

In [ ]:
linestyles = ['solid', (0, (3, 10, 1, 10)), 'dotted']
palette_1 = sns.color_palette("colorblind", 3)
font = 11
fig, axes = plt.subplots( 1,1, sharey = True, figsize = (7,7))
a =  0
for i in range(len(at_risk_df_list)):
    if i %2 == 0:

       legend = False
       sns.lineplot(at_risk_df_list[i + 1]['Step'], at_risk_df_list[i + 1]['Damage_coeff'] * at_risk_df_list[i + 1]['House_value'] / at_risk_df_list[i + 1]['Wage'] , hue = at_risk_df_list[i + 1]['Education'], palette =palette_1, ci=95, linestyle = linestyles[a], legend=legend )
       sns.lineplot(at_risk_df_list_homo[a]['Step'], at_risk_df_list_homo[a]['Damage_coeff'] * at_risk_df_list_homo[a]['House_value'] / at_risk_df_list_homo[a]['Wage'], color = 'darkred', ci=95, linestyle = linestyles[a] )

       a += 1
plt.plot(0,0, label = "Low adaptive capacity", linestyle =linestyles[0], color = palette_1[0] )
plt.plot(0,0, label = "Medium adaptive capacity", linestyle =linestyles[0], color = palette_1[1] )
plt.plot(0,0, label = "High adaptive capacity", linestyle =linestyles[0], color = palette_1[2] )
plt.plot(0,0, label = "Homogenous adaptive capacity", linestyle =linestyles[0], color = 'darkred' )
plt.plot(0,0, label = "No adaptation - Heterogenous", linestyle =linestyles[0], color = 'black' )
plt.plot(0,0, label = "EU - Heterogenous", linestyle =linestyles[1], color = 'black' )
plt.plot(0,0, label = "Extended PMT - Heterogenous", linestyle =linestyles[2], color = 'black' )
plt.axvline(x = 100, linestyle = 'dashed')
plt.axvline(x = 140, linestyle = 'dashed')
plt.ylabel('Potential damage normalized by income', fontsize = font)
plt.xlabel('Step', fontsize = font)
plt.legend( loc = 'best',bbox_to_anchor=(0.5, 0.6), fontsize = font -1)

In [ ]:
at_risk_df_list_homo

In [ ]:
linestyles = ['solid', (0, (3, 10, 1, 10)), 'dotted']
palette_1 = sns.color_palette("colorblind", 3)
palette_2 = sns.color_palette("colorblind", 1)
font = 11
fig, axes = plt.subplots( 1,2, sharey = True, figsize = (10,5))
a =  0
for i in range(len(at_risk_df_list)):
    if i %2 == 0 and i < 2:

       legend = False
       sns.stackplot()
       #sns.lineplot(at_risk_df_list[i + 1]['Step'], at_risk_df_list[i + 1]['Damage_coeff'] * at_risk_df_list[i + 1]['House_value'] / at_risk_df_list[i + 1]['Wage'] , hue = at_risk_df_list[i + 1]['Education'], palette =palette_1, ci=95, linestyle = linestyles[a], legend=legend, ax= axes[0] )
       #sns.lineplot(at_risk_df_list_homo[a]['Step'], at_risk_df_list_homo[a]['Damage_coeff'] * at_risk_df_list_homo[a]['House_value'] / at_risk_df_list_homo[a]['Wage'], palette =palette_2, ci=95, linestyle = linestyles[a], legend=legend, ax= axes[0] )
       axes[0].axvline(x = 100, linestyle = 'dashed')
       axes[0].axvline(x = 140, linestyle = 'dashed')
       a += 1
    if i %2 == 0 and i >= 2:
       legend = False
       sns.lineplot(at_risk_df_list[i + 1]['Step'], at_risk_df_list[i + 1]['Damage_coeff'] * at_risk_df_list[i + 1]['House_value'] / at_risk_df_list[i + 1]['Wage'] , hue = at_risk_df_list[i + 1]['Education'], palette =palette_1, ci=95, linestyle = linestyles[a], legend=legend, ax= axes[1] )
       sns.lineplot(at_risk_df_list_homo[a]['Step'], at_risk_df_list_homo[a]['Damage_coeff'] * at_risk_df_list_homo[a]['House_value'] / at_risk_df_list_homo[a]['Wage'], palette =palette_2, ci=95, linestyle = linestyles[a], legend=legend, ax= axes[1] )
       axes[1].axvline(x = 100, linestyle = 'dashed')
       axes[1].axvline(x = 140, linestyle = 'dashed')
       a += 1
plt.plot(0,0, label = "Low adaptive capacity", linestyle =linestyles[0], color = palette_1[0] )
plt.plot(0,0, label = "Medium adaptive capacity", linestyle =linestyles[0], color = palette_1[1] )
plt.plot(0,0, label = "High adaptive capacity", linestyle =linestyles[0], color = palette_1[2] )
plt.plot(0,0, label = "No adaptation - Heterogenous", linestyle =linestyles[0], color = 'black' )
plt.plot(0,0, label = "EU - Heterogenous", linestyle =linestyles[1], color = 'black' )
plt.plot(0,0, label = "Extended PMT - Heterogenous", linestyle =linestyles[2], color = 'black' )

axes[0].set_ylabel('Potential damage normalized by income', fontsize = font)
plt.xlabel('Step', fontsize = font)
plt.legend( loc = 'best', fontsize = font -1)

#,bbox_to_anchor=(0.5, 0.6)

In [ ]:
dataset_big = pd.concat([at_risk_df_list[1], at_risk_df_list[3], at_risk_df_list[5]])

In [ ]:
a  = dataset_big.loc[dataset_big['Theory'] == 'No adaptation']

In [ ]:
titles = ['Low', 'Med', 'High']
linestyles = ['solid', (0, (3, 10, 1, 10)), 'dotted']
palette_1 = sns.color_palette("colorblind", 3)
font = 11
fig, axes = plt.subplots( 1,3, sharey = True, figsize = (15,5))
a =  0
for i in range(3):
   # if i %2 == 0:
       dataset_trial = dataset_big.loc[dataset_big['Education'] == a + 2]
       dataset_no_cca = dataset_trial.loc[dataset_trial['Theory'] == 'No adaptation']
       legend = False
       sns.lineplot(dataset_trial['Step'],  (dataset_trial['Damage_coeff'] * dataset_trial['House_value'] / dataset_trial['Wage']) , hue = dataset_trial['Theory'], palette =palette_1, ci=95, linestyle = 'solid', legend=legend, ax = axes[a] )
       # sns.lineplot(dataset_no_cca['Step'],  (dataset_no_cca['Damage_coeff'] * dataset_no_cca['House_value'] / dataset_no_cca['Wage']) , hue = dataset_trial['Theory'], palette =palette_1, ci=95, linestyle = 'solid', legend=legend, ax = axes[a] )
       axes[a].axvline(x = 100, linestyle = 'solid')
       axes[a].axvline(x = 140, linestyle = 'solid')
       axes[a].axhline(y = 0, linestyle = 'dotted')
       axes[a].set_title(titles[a], fontsize = font + 3)
       axes[0].set_ylabel('Damage reduction normalized by income', fontsize = font + 2)
       axes[a].set_xlabel('Step', fontsize = font + 2)
       a += 1
#plt.plot(0,0, label = "Low adaptive capacity", linestyle =linestyles[0], color = palette_1[0] )
#plt.plot(0,0, label = "Medium adaptive capacity", linestyle =linestyles[0], color = palette_1[1] )
#plt.plot(0,0, label = "High adaptive capacity", linestyle =linestyles[0], color = palette_1[2] )
#plt.plot(0,0, label = "No adaptation - Heterogenous", linestyle =linestyles[0], color = 'black' )
#plt.plot(0,0, label = "EU - Heterogenous", linestyle =linestyles[1], color = 'black' )
#plt.plot(0,0, label = "Extended PMT - Heterogenous", linestyle =linestyles[2], color = 'black' )

plt.ylabel('Potential damage normalized by income', fontsize = font)
plt.xlabel('Step', fontsize = font)
plt.legend( loc = 'best',bbox_to_anchor=(0.5, 0.6), fontsize = font -1)

In [ ]:
red_at_risk_df_list = []
red_at_risk_df_list_homo = []
homo = 0
for i in range(len(micro_dfs)):
    at_risk_df =  micro_dfs[i].loc[(micro_dfs[i]['At_risk'] == True) & (micro_dfs[i]['Education'] != 1) & (micro_dfs[i]['Step'] > 95) &  (micro_dfs[i]['Step'] < 200) &  (micro_dfs_homo[i]['AgentID'] % 10 == 0)]# & (datasets[i]['Flooded'] == True)]
    #at_risk_df['Theory'] = theories[i]
    red_at_risk_df_list.append(at_risk_df)
    
    if i % 2 == 0:

        at_risk_df_homo =  micro_dfs_homo[homo].loc[(micro_dfs_homo[homo]['At_risk'] == True) & (micro_dfs_homo[homo]['Education'] != 1) & (micro_dfs_homo[homo]['Step'] >1 ) & (micro_dfs[i]['Step'] > 95) &  (micro_dfs[i]['Step'] < 200) &  (micro_dfs_homo[homo]['AgentID'] % 10 == 0) ]
        #at_risk_df_homo['Theory'] = theories[homo]
        red_at_risk_df_list_homo.append(at_risk_df_homo)

        homo+=1 

In [ ]:
linestyles = ['solid', (0, (3, 10, 1, 10)), 'dotted']
palette_1 = sns.color_palette("colorblind", 3)
font = 11
fig, axes = plt.subplots( 1,1, sharey = True, figsize = (7,7))
sns.barplot(  data = red_at_risk_df_list[0] , y = red_at_risk_df_list[0]['House_wage_ratio'], x = red_at_risk_df_list[0]["Education"], palette =palette_1)
plt.plot(0,0, label = "Low adaptive capacity", linestyle =linestyles[0], color = palette_1[0] )
plt.plot(0,0, label = "Medium adaptive capacity", linestyle =linestyles[0], color = palette_1[1] )
plt.plot(0,0, label = "High adaptive capacity", linestyle =linestyles[0], color = palette_1[2] )
plt.ylabel('Av. House value normalized by income', fontsize = font + 2)
plt.xlabel('Education level', fontsize = font + 2)
plt.xticks([])
plt.legend( loc = 'best',bbox_to_anchor=(0.25, 0.6), fontsize = font -1)

In [ ]:
titles = ['No adaptation - Heterogenous', 'EU - Heterogenous', 'Extended PMT - Heterogenous']
linestyles = ['solid', (0, (3, 10, 1, 10)), 'dotted']
palette_1 = sns.color_palette("colorblind", 3)
font = 11
fig, axes = plt.subplots( 1,3, figsize = (15,5))
a =  0
for i in range(len(red_at_risk_df_list)):
    if i %2 == 0:
       print(i)
       legend = False
       sns.lineplot(red_at_risk_df_list_homo[a]['Step'], (red_at_risk_df_list_homo[a]['Net worth']- red_at_risk_df_list_homo[a]['Monetary_damage'])/red_at_risk_df_list_homo[a]['Wage'],linestyle = 'solid' ,palette ='darkred', ci=95, legend=legend, ax= axes[a])
       sns.lineplot(red_at_risk_df_list[i + 1]['Step'], (red_at_risk_df_list[i + 1]['Net worth']- red_at_risk_df_list[i + 1]['Monetary_damage'])/red_at_risk_df_list[i + 1]['Wage'], hue = red_at_risk_df_list[i + 1]['Education'],linestyle = 'solid' ,palette =palette_1, ci=95, legend=legend, ax= axes[a])
       sns.lineplot(red_at_risk_df_list[i]['Step'], (red_at_risk_df_list[i]['Net worth']- red_at_risk_df_list[i]['Monetary_damage'])/red_at_risk_df_list[i]['Wage'], hue = red_at_risk_df_list[i]['Education'], palette =palette_1, ci=None, linestyle = 'dashed', legend=legend, ax= axes[a])
       axes[a].axvline(x = 100, linestyle = 'solid')
       axes[a].axvline(x = 140, linestyle = 'solid')
       axes[a].axhline(y = 0, linestyle = 'dotted')
       axes[a].set_title(titles[a], fontsize = font + 3)
       axes[0].set_ylabel('Liquid resources to wage ratio', fontsize = font + 2)
       axes[a].set_xlabel('Step', fontsize = font + 2)
       if a > 0:
           axes[a].set_ylim([-1, 10])

       a += 1


plt.plot(100,0, label = "Low adaptive capacity", linestyle =linestyles[0], color = palette_1[0] )
plt.plot(100,0, label = "Medium adaptive capacity", linestyle =linestyles[0], color = palette_1[1] )
plt.plot(100,0, label = "High adaptive capacity", linestyle =linestyles[0], color = palette_1[2] )
plt.plot(100,0, label = "Homogenous adaptive capacity", linestyle =linestyles[0], color = 'darkred' )
plt.plot(100,0, label = "Two floods", linestyle ='solid', color = 'black' )
plt.plot(100,0, label = "Zero liquid resources", linestyle ='dotted', color = 'black' )
plt.plot(100,0, label = "Counterfactual" "\n" "with no floods", linestyle ='dashed', color = 'black' )

#plt.plot(0,0, label = "No adaptation", linestyle =linestyles[0], color = 'black' )
#plt.plot(0,0, label = "Expected utility (EU)", linestyle =linestyles[1], color = 'black' )
#plt.plot(0,0, label = "Protection Motivation Theory (PMT) ", linestyle =linestyles[2], color = 'black' )


plt.legend( loc='center', bbox_to_anchor =(-0.5, 0.2), fontsize = font)

In [ ]:
at_risk_df_list[0]

In [ ]:
fig, axes = plt.subplots( 1,3, sharey = True, figsize = (15,6))
a =  0
for i in range(len(at_risk_df_list)):
    if i %2 == 0:
       legend = False 
       sns.lineplot(at_risk_df_list[i + 1]['Step'], (at_risk_df_list[i + 1]['Net worth']- at_risk_df_list[i + 1]['Monetary_damage'])/at_risk_df_list[i + 1]['House_wage_ratio'], hue = at_risk_df_list[i + 1]['Education'], ax= axes[a])
       sns.lineplot(at_risk_df_list[i]['Step'], (at_risk_df_list[i]['Net worth']- at_risk_df_list[i]['Monetary_damage'])/at_risk_df_list[i]['House_wage_ratio'], hue = at_risk_df_list[i]['Education'],linestyle = '--', ax= axes[a])
       a += 1

In [ ]:
fig, axes = plt.subplots( 1,3, sharey = True, figsize = (15,6))
a =  0
for i in range(len(at_risk_df_list)):
    if i %2 == 0:
       legend = False 
       sns.lineplot(at_risk_df_list[i + 1]['Step'], (at_risk_df_list[i + 1]['Net worth']- at_risk_df_list[i + 1]['Monetary_damage'])/at_risk_df_list[i + 1]['House_wage_ratio'], hue = at_risk_df_list[i + 1]['Education'], ax= axes[a])
       sns.lineplot(at_risk_df_list[i]['Step'], (at_risk_df_list[i]['Net worth']- at_risk_df_list[i]['Monetary_damage'])/at_risk_df_list[i]['House_wage_ratio'], hue = at_risk_df_list[i]['Education'],linestyle = '--', ax= axes[a])
       a += 1

In [ ]:
fig, axes = plt.subplots( 2,2, sharey = True, figsize = (10,10))
for i in range(len(at_risk_df_list)):
    j = 0
    legend = True
    if i > 1:
        j = 1
    if i > 0:   
       legend = False
       sns.lineplot(at_risk_df_list[i]['Step'], (at_risk_df_list[i]['Net worth']- at_risk_df_list[i]['Monetary_damage']), hue = at_risk_df_list[i]['Education'], ax= axes[i%2][j])

In [ ]:
stop

sns.lineplot(df_flooded['Step'], df_flooded['Monetary_damage']/ df_flooded['House_wage_ratio'], hue = df_flooded['Education'])

In [ ]:

node_label = ["A1", "A2", "A3", "A4","A5", "B1", "B2","B3", "B4", "B5","C1", "C2", "C3", "C4","C5" ]
node_dict = {y:x for x, y in enumerate(node_label)}
node_dict

In [ ]:

source = []
for i in range(51):
    if i < 5:
        source.append('A1')
    elif i > 5 and i <=10:
        source.append('A2')
    elif i > 10 and i <= 15:
        source.append('A3')
    elif i > 15 and i <= 20:
        source.append('A4')
    elif i > 20 and i <= 25:
        source.append('A5')
    elif i > 25 and i <= 30:
        source.append('B1')
    elif i > 30 and i <=35:
        source.append('B2')
    elif i > 35 and i <= 40:
        source.append('B3')
    elif i > 40 and i <= 45:
        source.append('B4')
    elif i > 45 and i <= 50:
        source.append('A5')


target_1 = []
for i in range(25):
    if i % 5 == 0:
        target_1.append('B1')
    elif i%5 == 1:
        target_1.append('B2')
    elif i%5 == 2:
        target_1.append('B3')
    elif i%5 == 3:
        target_1.append('B4')
    elif i%5 == 4:
        target_1.append('B5')
target_2 = []
for i in range(25):
    if i % 5 == 0:
        target_2.append('C1')
    elif i%5 == 1:
        target_2.append('C2')
    elif i%5 == 2:
        target_2.append('C3')
    elif i%5 == 3:
        target_2.append('C4')
    elif i%5 == 4:
        target_2.append('C5')   

#target = ['B1','B2','B3','B1','B2','B3','C1','C1','C2','C1','C2'] 
#values = [ 10, 5, 15, 5, 20, 45, 15, 20, 5, 30, 30 ]
target = target_1 + target_2

In [ ]:
df = pd.DataFrame(target)
df_2 = pd.DataFrame(source)

In [ ]:
df_2

In [ ]:
len(target), len(source)

In [ ]:
pre_flood_df_list = []
for i in range(len(at_risk_df_list)):
    pre_flood_df =  at_risk_df_list[i].loc[at_risk_df_list[i]['Step'] == 99]
    pre_flood_df['Quintile_pre_flood'] = None
    pre_flood_df_list.append(pre_flood_df)

In [ ]:
quintiles_list = []
for i in range(len(pre_flood_df_list)):
 #+ benchmark['House_wage_ratio'] 
    quintiles = pre_flood_df_list[i]['Net worth'].quantile([0.2,0.4,0.6,0.8])
    quintiles_list.append(quintiles)

In [ ]:
datasets_quintiles_list = []
for i in range(len(pre_flood_df_list)):
    dataset = pre_flood_df_list [i]
    variable = 'Quintile_pre_flood'
    variable_2 ='Net worth'
    a = quintiles_list[i]
    for x,line in dataset.iterrows():
    # print(x)
        if line[variable_2] <= a.iloc[0]:
            dataset[variable][x] = 1
        elif line[variable_2] > a.iloc[0] and line[variable_2] <= a.iloc[1] :
            dataset[variable][x] = 2
        elif line[variable_2] > a.iloc[1] and line[variable_2] <= a.iloc[2] :
            dataset[variable][x] = 3
        elif line[variable_2] > a.iloc[2] and line[variable_2] <= a.iloc[3] :
            dataset[variable][x] = 4
        elif line[variable_2] > a.iloc[3] :
            dataset[variable][x] = 5

In [ ]:
dataset

In [ ]:
post_flood_df_list = []
for i in range(len(at_risk_df_list)):
    post_flood_df =  at_risk_df_list[i].loc[at_risk_df_list[i]['Step'] == 110]
    post_flood_df['Quintile_post_flood'] = None
    post_flood_df_list.append(post_flood_df)

In [ ]:
post_2floods_df_list = []
for i in range(len(at_risk_df_list)):
    post_flood_df =  at_risk_df_list[i].loc[at_risk_df_list[i]['Step'] == 150]
    post_flood_df['Quintile_post_2floods'] = None
    post_2floods_df_list.append(post_flood_df)

In [ ]:

quintiles_list_post = []
for i in range(len(post_flood_df_list)):
 #+ benchmark['House_wage_ratio'] 
    quintiles = post_flood_df_list[i]['Net worth'].quantile([0.2,0.4,0.6,0.8])
    quintiles_list_post.append(quintiles)

In [ ]:

quintiles_list_post_2 = []
for i in range(len(post_2floods_df_list)):
 #+ benchmark['House_wage_ratio'] 
    quintiles = post_2floods_df_list[i]['Net worth'].quantile([0.2,0.4,0.6,0.8])
    quintiles_list_post_2.append(quintiles)

In [ ]:

for i in range(len(post_flood_df_list)):
    dataset = post_flood_df_list [i]
    variable = 'Quintile_post_flood'
    variable_2 ='Net worth'
    a = quintiles_list_post[i]
    for x,line in dataset.iterrows():
    # print(x)
        if line[variable_2] <= a.iloc[0]:
            dataset[variable][x] = 1
        elif line[variable_2] > a.iloc[0] and line[variable_2] <= a.iloc[1] :
            dataset[variable][x] = 2
        elif line[variable_2] > a.iloc[1] and line[variable_2] <= a.iloc[2] :
            dataset[variable][x] = 3
        elif line[variable_2] > a.iloc[2] and line[variable_2] <= a.iloc[3] :
            dataset[variable][x] = 4
        elif line[variable_2] > a.iloc[3] :
            dataset[variable][x] = 5

In [ ]:

for i in range(len(post_2floods_df_list)):
    dataset = post_2floods_df_list [i]
    variable = 'Quintile_post_2floods'
    variable_2 ='Net worth'
    a = quintiles_list_post_2[i]
    for x,line in dataset.iterrows():
    # print(x)
        if line[variable_2] <= a.iloc[0]:
            dataset[variable][x] = 1
        elif line[variable_2] > a.iloc[0] and line[variable_2] <= a.iloc[1] :
            dataset[variable][x] = 2
        elif line[variable_2] > a.iloc[1] and line[variable_2] <= a.iloc[2] :
            dataset[variable][x] = 3
        elif line[variable_2] > a.iloc[2] and line[variable_2] <= a.iloc[3] :
            dataset[variable][x] = 4
        elif line[variable_2] > a.iloc[3] :
            dataset[variable][x] = 5

In [ ]:
post_2floods_df_list[0]

In [ ]:
df_num = 5

In [ ]:
pre_flood_df_list[df_num] = pre_flood_df_list[df_num].set_index('AgentID')
post_flood_df_list[df_num] = post_flood_df_list[df_num].set_index('AgentID')
post_2floods_df_list[df_num] = post_2floods_df_list[df_num].set_index('AgentID')

In [ ]:
pre_flood_df_list[df_num]['Quintile_post_flood'] = post_flood_df_list[df_num]['Quintile_post_flood'] 
pre_flood_df_list[df_num]['Quintile_post_2floods'] = post_flood_df_list[df_num]['Quintile_post_2floods'] =  post_2floods_df_list[df_num]['Quintile_post_2floods']

In [ ]:
pre_flood_df_list[df_num] = pre_flood_df_list[df_num].dropna()

In [ ]:
values_1 = []
values_2 = []
pre_quint = 1
for i in range(25):
    
    post_quint = 1 + i%5
    if i > 4 and post_quint == 1:
        pre_quint += 1
    #if post_quint%5 == 0:
     #   pre_quint +=1 
    print(pre_quint, post_quint)
    a = pre_flood_df_list[df_num].loc[(pre_flood_df_list[df_num]['Quintile_pre_flood'] == pre_quint) & (pre_flood_df_list[df_num]['Quintile_post_flood'] == post_quint)]
    b = pre_flood_df_list[df_num].loc[(pre_flood_df_list[df_num]['Quintile_post_flood'] == pre_quint) & (pre_flood_df_list[df_num]['Quintile_post_2floods'] == post_quint)]
    values_1.append(len(a))
    values_2.append(len(b))
values = values_1 + values_2


In [ ]:
values

In [ ]:

import pandas as pd
import numpy as np
from webcolors import hex_to_rgb
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
init_notebook_mode(connected=True)

In [ ]:
target

In [ ]:

source_node = [node_dict[x] for x in source]
target_node = [node_dict[x] for x in target]

In [ ]:

import plotly.graph_objects as go # Import the graphical object

fig = go.Figure( 
    data=[go.Sankey( # The plot we are interest
        # This part is for the node information
        node = dict( 
            label = node_label
        ),
        # This part is for the link information
        link = dict(
            source = source_node,
            target = target_node,
            value = values
        ))])

# With this save the plots 
plot(fig,
     image_filename='sankey_plot_1', 
     image='png', 
     image_width=1000, 
     image_height=600
)
# And shows the plot
fig.show()

In [ ]:
fig = go.Figure( 
    data=[go.Sankey( # The plot we are interest
        # This part is for the node information
        node = dict( 
            label = node_label
        ),
        # This part is for the link information
        link = dict(
            source = source_node,
            target = target_node,
            value = values
        ))])

# With this save the plots 
plot(fig,
     image_filename='sankey_plot_1', 
     image='png', 
     image_width=1000, 
     image_height=600
)
# And shows the plot
fig.show()

In [ ]:
fig = go.Figure( 
    data=[go.Sankey( # The plot we are interest
        # This part is for the node information
        node = dict( 
            label = node_label
        ),
        # This part is for the link information
        link = dict(
            source = source_node,
            target = target_node,
            value = values
        ))])

# With this save the plots 
plot(fig,
     image_filename='sankey_plot_1', 
     image='png', 
     image_width=1000, 
     image_height=600
)
# And shows the plot
fig.show()

In [ ]:
fig = go.Figure( 
    data=[go.Sankey( # The plot we are interest
        # This part is for the node information
        node = dict( 
            label = node_label
        ),
        # This part is for the link information
        link = dict(
            source = source_node,
            target = target_node,
            value = values
        ))])

# With this save the plots 
plot(fig,
     image_filename='sankey_plot_1', 
     image='png', 
     image_width=1000, 
     image_height=600
)
# And shows the plot
fig.show()

In [ ]:
fig = go.Figure( 
    data=[go.Sankey( # The plot we are interest
        # This part is for the node information
        node = dict( 
            label = node_label
        ),
        # This part is for the link information
        link = dict(
            source = source_node,
            target = target_node,
            value = values
        ))])

# With this save the plots 
plot(fig,
     image_filename='sankey_plot_1', 
     image='png', 
     image_width=1000, 
     image_height=600
)
# And shows the plot
fig.show()